### Introduction to Google Earth Engine's Python API through Google Colab.  

Contents:

- Part A: Set up your notebook to access the python API and all required packages
- Part B: Import and filter an image collection
- Part C: Print statements
- Part D: Vizualize an image 
- Part E: Calculate NDVI 4 ways




---



# PART A

### Imports, installation, authentication, initialization

Our first step is to get the notebook set up. 

In [1]:
# Install geemap library so that we can use it to view images on an interactive map
## You may get a runtime warning - you can ignore that
!pip install geemap
!pip install geopandas

# the exclamation mark indicates a shell command, rather than notebook command

     |████████████████████████████████| 471kB 6.6MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 1.6MB 39.9MB/s 
     |████████████████████████████████| 225kB 38.9MB/s 
     |████████████████████████████████| 3.3MB 31.0MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 1.2MB 30.9MB/s 
     |████████████████████████████████| 1.3MB 19.6MB/s 
     |████████████████████████████████| 143kB 54.5MB/s 
     |████████████████████████████████| 552kB 35.9MB/s 
     |████████████████████████████████| 389kB 36.4MB/s 
     |████████████████████████████████| 122kB 46.8MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 122kB 50.0MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 81kB 9.9MB/s 
  C

     |████████████████████████████████| 1.0MB 8.2MB/s 
     |████████████████████████████████| 6.6MB 12.9MB/s 
     |████████████████████████████████| 15.4MB 199kB/s 


In [2]:
# Import the necessary libraries
import ee
import numpy as np
import geemap.eefolium as geemap
import geopandas as geopandas
import json
import pandas as pd

In [14]:
# Authenticate and initialize this instance of GEE in Google Colab
## Follow the prompts and fill in authentication code
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=tJPxbhvOgyM_0gTw3bYsaiRJK0Zoho-2cJ3w2shIsl4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6u6nU7iIrGJ29KoTkELtxlwmfUjT_O5BbP1hKO6YiZj4brYyGVN0M

Successfully saved authorization token.




---



# Part B: Import and Filter Imagery

## Now, let's start working with GEE 

First, we will grab one specific image.  In this case, *we know the assetID of this image.*

In [15]:
image1 = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_015028_20200814')
print(image1.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7791, 7901], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 426585, 0, -30, 5215515]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7791, 7901], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 426585, 0, -30, 5215515]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7791, 7901], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 426585, 0, -30, 5215515]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7791, 7901], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 426585, 0, -30, 5215515]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7791, 7901], 'crs': 'EPSG:32618', 'crs_transform': [30, 0, 4265

Let's plot that one image on the map. 

- we will need to tell GEE how to visualize the data so we create a "visualization parameter". 

In [16]:
Map = geemap.Map()

# we need to create a visualization parameter variable  
# This will tell GEE which bands to display, and what colour stretch to use
# Set the L8 true colour vis params
L8_TC_vis = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

Map.addLayer(image1,L8_TC_vis,'image LC08_015028_20200814')

# Let's draw a point on the map as well  
AlfredPt = ee.Geometry.Point(-74.819, 45.4969)
Map.addLayer(AlfredPt)
Map.addLayerControl()
Map.centerObject(AlfredPt,10)

Map


But what happens if you don't know the assetID of the image you want? Or you want multiple images? 

Let's use the filter functions to pick all suitable images for an AOI. 

In [17]:
# Import the landsat-8 tier 1 surface reflectance as a variable
l8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') 

# Filter for images within our date range that overlap our point
# Sort by cloud cover and choose least cloudy (first)
l8SR_Jul2019 = l8SR.filterBounds(AlfredPt) \
                      .filterDate('2020-08-01', '2020-08-31')





---



# Part C: Print statements

So...what did that do?  

Let's see what this new variable contains.  

In [18]:
# For this, we will use a print statement
print(l8SR_Jul2019.getInfo())

{'type': 'ImageCollection', 'bands': [], 'id': 'LANDSAT/LC08/C01/T1_SR', 'version': 1624291867209337, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'visualization_1_bands': 'B5,B4,B3', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png', 'visualization_1_max': '30000.0', 'description': '<p>This dataset is the atmospherically corrected\nsurface reflectance from  the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and two thermal infrared (TIR) bands processed to orthorectified\nbrightness temperature</p><p>These data have been atmospherically corrected using\n<a href="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1368_L8_C1-LandSurfaceReflectanceCode-LASRC_ProductGuide-v3.pdf">LaSRC</a>\nand includes a cloud, shadow, water and snow mask produced using\n<a href

This is pretty messy.  Instead, let's create some functions that we can use to easily understand our images/image collections


In [19]:
# these print statments were written by Adam Mohiuddin.  Please feel free to use and edit but acknowledge the original source.
# all of these functions require the image collection's properties to be stored json format -- do this using getInfo() 
# Print out all the band ids for an image json
def printImageBands(image):
  print('Bands of example image:')
  for band in range(len(image['bands'])):
    print(image['bands'][band]['id'])
  print('\n')

# Print the size of a json image collection
def printCollectionSize(collection):
  print('Size of image collection:')
  print(len(collection['features']))
  print('\n')

# Get an image json object from a json image collection based on an index value
def getImageFromCollection(collection, index=0):
  return collection['features'][index]

# Print all image ids from a json image collection
def printImageNames(collection):
  print('All images in image collection:')
  for img in range(len(collection['features'])):
    print(str(img) + ': ' + collection['features'][img]['id'])
  print('\n')

In [20]:
# Now let's use those print functions! 
# First, we need to use the getInfo function to get all the information about our image Collection into json format
stack_json = l8SR_Jul2019.getInfo()

# Then, we can run our neat print functions on the new variable
# printCollectionSize tells us how many images are in our image collection
printCollectionSize(stack_json)
# printImageNames shows all the image ids from every image in the collection
printImageNames(stack_json)

# If we want to look at a specific image in the collection, we need to first grab an image from our json variable and put it in a new variable
img_json = getImageFromCollection(stack_json)

# Then we can look at what bands are in this image
printImageBands(img_json)

Size of image collection:
2


All images in image collection:
0: LANDSAT/LC08/C01/T1_SR/LC08_015028_20200814
1: LANDSAT/LC08/C01/T1_SR/LC08_015028_20200830


Bands of example image:
B1
B2
B3
B4
B5
B6
B7
B10
B11
sr_aerosol
pixel_qa
radsat_qa




We can see that we have 4 images the the collection.  Let's filter this further.

In [21]:
# Let's take a look at the type of object we are working with:
print("l8SR_Jul2019 is an", l8SR_Jul2019.name())

l8SR_Jul2019 is an ImageCollection


In [22]:
# Let's get the least cloudy image in the collection
# We will sort using the CLOUD_COVER property from the metadata, and then take the first image
l8SR_Jul2019_LC = l8SR_Jul2019 \
                    .filterDate('2020-08-01', '2020-08-31') \
                    .sort("CLOUD_COVER") \
                    .first()
  
print("l8SR_Jul2019_LC is an", l8SR_Jul2019_LC.name())

l8SR_Jul2019_LC is an Image


Ok! so now we have a single image!  Let's move on to visualization. 



---



# Part D Vizualize an Image

In [26]:
# Let's access some data on our google drive. 


# Mounting your google drive
from google.colab import drive
drive.mount('/content/drive')

#if the path to your data is different than mine, you should change it.
Alfred = '/content/drive/MyDrive/CSRS2021_GEEWorkshop_ForStudents/datasets/AlfredBog.shp' # all the rest of the parts of the shapefile will be read as well. 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
Map = geemap.Map()

# We need to create a visualization parameter variable  
# This will tell GEE which bands to display, and what colour stretch to use
# Set the L8 true colour vis params
L8_TC_vis = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

# a very simple visualization parameter for our shapefile
SHPViz = {
    'color': 'red' 
    }


shpdata = geemap.shp_to_ee(Alfred)
Map.addLayer(l8SR_Jul2019_LC,L8_TC_vis,'l8SR_Jul2019_LC')
Map.addLayer(shpdata, SHPViz, 'shpdata')

Map.addLayerControl()
Map.centerObject(shpdata,10)

Map



---



#Part E: Let's start doing some remote sensing! 
We are going to calculate NDVI 4 different ways.

## Method 1: Calculate NDVI using simple Band Math

Multispectral indices are calculated using simple **band math**. 

Recall that the Normalized Difference Vegetation Index (NDVI) is:

```
(NIR - RED) / (NIR + RED)
```
We can calculate any multi-spectral index as long as we are using a sensor that  has the bands we need. For example, we can easily calculate NDVI for Landsat 8 using bandmath, where Band 5 is the NIR band and Band 4 is the Red band:

```
NDVI = image.select('B5').subtract(image.select('B4')) 
          .divide(image.select('B5').add(image.select('B4')))
```

The `image.subtract(image2)`, `image.add(image2)`, and `image.divide(image2)` are all math functions that operate on two images. 

They are the equivalent of:

`image - image2`

`image + image2`

`image / image2`

These image math functions can be used to do any kind of band math you need to calculate a given index, or to calculate other kinds of band relationships.

The `image.select()` function selects specified bands for an image as either a single band as with `image.select('B5')` or a list of bands like `image.select(['B5','B4'])`. Note that when you want to select multiple bands, you need to put them in a list with square brackets.

By using `image.select(Band)`, we are making sure the raster math is done only with the bands we want to use within *the same image*, rather than with every band in two different images.

In [ ]:
NDVI = l8SR_Jul2019_LC.select('B5').subtract(l8SR_Jul2019_LC.select('B4')) \
               .divide(l8SR_Jul2019_LC.select('B5').add(l8SR_Jul2019_LC.select('B4'))) \
               .rename('ndvi')
print('NDVI Bands:\n',NDVI.get('system:band_names').getInfo())


In [ ]:
Map = geemap.Map()

NDVI_vis = {'min': 0.25, 'max': 1, 'palette': ['FFFFFF', '006400']}

Map.addLayer(NDVI, NDVI_vis, 'NDVI')
Map.addLayerControl()
Map.centerObject(AlfredPt,12)

Map



---



## Method 2: Calculate NDVI using a built-in function

In [ ]:
ndvi = l8SR_Jul2019_LC.normalizedDifference(['B5', 'B4']).rename('ndvi') #note: renamed band to 'ndvi' as default is 'nd'

Map = geemap.Map()

NDVI_vis = {'min': 0.25, 'max': 1, 'palette': ['FFFFFF', '006400']}

Map.addLayer(ndvi, NDVI_vis, 'NDVI')
Map.addLayerControl()
Map.centerObject(AlfredPt,12)

Map


## Method 3: Map over each image in a collection and produce NDVI for each

In [ ]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename("ndvi")
  return ndvi

l8SR_Jul2019_ndvi= l8SR_Jul2019.map(addNDVI)
print(l8SR_Jul2019_ndvi.getInfo())

{'type': 'ImageCollection', 'bands': [], 'id': 'LANDSAT/LC08/C01/T1_SR', 'version': 1624218086844841, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'visualization_1_bands': 'B5,B4,B3', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png', 'visualization_1_max': '30000.0', 'description': '<p>This dataset is the atmospherically corrected\nsurface reflectance from  the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and two thermal infrared (TIR) bands processed to orthorectified\nbrightness temperature</p><p>These data have been atmospherically corrected using\n<a href="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1368_L8_C1-LandSurfaceReflectanceCode-LASRC_ProductGuide-v3.pdf">LaSRC</a>\nand includes a cloud, shadow, water and snow mask produced using\n<a href

In [ ]:
# Print section
# First, we need to use the getInfo function to get all the information about our image Collection into json format
stack_json = l8SR_Jul2019_ndvi.getInfo()

# Then, we can run our neat print functions on new variable.
# printCollectionSize tells us how many images are in our image collection
printCollectionSize(stack_json)

# If we want to look at a specific image in the collection, we need to first grab an image from our json variable and put it in a new variable
img_json = getImageFromCollection(stack_json)

# Then we can look at what bands are in this image
printImageBands(img_json)

## Method 4: Map over each image in an image collection and add an NDVI band to each 
Now let's create a new function that calculates NDVI and adds it back to the original image collection! 

In [ ]:
def addNDVI2(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
  return image.addBands(ndvi)

withNDVI = l8SR_Jul2019.map(addNDVI2);

In [ ]:
# Print section
# First, we need to use the getInfo function to get all the information about our image Collection into json format 
stack_json = withNDVI.getInfo()

# Then, we can run our neat print functions on new variable.
# printCollectionSize tells us how many images are in our image collection
# printImageNames shows all the image ids from every image in the collection
printCollectionSize(stack_json)
printImageNames(stack_json)

# If we want to look at a specific image in the collection, we need to first grab an image from our json variable and put it in a new variable
img_json = getImageFromCollection(stack_json)

# Then we can look at what bands are in this image
printImageBands(img_json)